In [ ]:
import sys
sys.path.append('../')
from src.model_trainer import load_bert_model, train_model
import torch
import optuna




with open('../data/train_inputs.pt', 'rb') as f:
    train_inputs = torch.load(f)
with open('../data/train_labels.pt', 'rb') as f:
    train_labels = torch.load(f)



def objective(trial):
    # Suggest values for the hyperparameters
    num_epochs = trial.suggest_int('num_epochs', 1, 10)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)

    # Train the model with these hyperparameters
    model = load_bert_model(num_labels=10)
    _, val_accuracy = train_model(model, train_inputs_split, train_labels_split, val_inputs_split, val_labels_split, num_epochs, batch_size, learning_rate)

    # The objective is to maximize the validation accuracy
    return val_accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters
print(study.best_params)

